In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
!mkdir -p ~/.kaggle
from google.colab import drive
from google.colab import files
# Upload your kaggle.json API key
uploaded = files.upload()

drive.mount('/content/drive')

Saving kaggle.json to kaggle.json
Mounted at /content/drive


In [ ]:
import os
import shutil
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/Machine-learning'


In [ ]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Machine-learning/hmnist_28_28_RGB.csv')

# Extract features and labels
x = data.drop(columns=['label']).values
y = data['label'].values

In [ ]:
# Preprocess the data, split, and normalize
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler()
x, y = oversample.fit_resample(x, y)
x = np.array(x).reshape(-1, 28, 28, 3)
x = (x - np.mean(x)) / np.std(x)
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=1)


In [ ]:
# Build the Inception model
from tensorflow.keras.layers import concatenate
def inception_module(x, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)

    conv3x3 = Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
    conv3x3 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3x3)

    conv5x5 = Conv2D(filters[3], (1, 1), padding='same', activation='relu')(x)
    conv5x5 = Conv2D(filters[4], (5, 5), padding='same', activation='relu')(conv5x5)

    pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool = Conv2D(filters[5], (1, 1), padding='same', activation='relu')(pool)

    output = concatenate([conv1x1, conv3x3, conv5x5, pool], axis=-1)
    return output

In [ ]:
input_layer = Input(shape=(28, 28, 3))
x = inception_module(input_layer, [64, 128, 128, 32, 32, 32])
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
output_layer = Dense(7, activation='softmax')(x)

In [ ]:
model = Model(inputs=input_layer, outputs=output_layer)


In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [ ]:
# Create data generators
train_generator = datagen.flow(X_train, Y_train, subset='training')
val_generator = datagen.flow(X_train, Y_train, subset='validation')

In [ ]:
# Train the model
history = model.fit(train_generator,
                    epochs=30,
                    validation_data=val_generator)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Accuracy:", accuracy)

Epoch 1/30
939/939 [==============================] - 1610s 2s/step - loss: 1.6635 - accuracy: 0.4356 - val_loss: 1.2442 - val_accuracy: 0.5075
Epoch 2/30
939/939 [==============================] - 1634s 2s/step - loss: 1.1926 - accuracy: 0.5363 - val_loss: 1.1247 - val_accuracy: 0.5620
Epoch 3/30
939/939 [==============================] - 1615s 2s/step - loss: 1.0979 - accuracy: 0.5806 - val_loss: 1.0392 - val_accuracy: 0.6011
Epoch 4/30
939/939 [==============================] - 1594s 2s/step - loss: 1.0305 - accuracy: 0.6079 - val_loss: 1.0130 - val_accuracy: 0.6203
Epoch 5/30
939/939 [==============================] - 1592s 2s/step - loss: 0.9770 - accuracy: 0.6290 - val_loss: 0.9335 - val_accuracy: 0.6490
Epoch 6/30
939/939 [==============================] - 1587s 2s/step - loss: 0.9353 - accuracy: 0.6485 - val_loss: 0.9520 - val_accuracy: 0.6375
Epoch 7/30
939/939 [==============================] - 1610s 2s/step - loss: 0.9023 - accuracy: 0.6617 - val_loss: 0.9236 - val_accuracy:

In [ ]:
# Plot training history
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
# Generate classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(Y_test, y_pred_classes))